## List avaliable amazon bedrock models

In [ ]:

"""
Lists the available Amazon Bedrock models.
"""
import logging
import json
import boto3


from botocore.exceptions import ClientError


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise


def list_granted_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models with granted access.

    :return: The list of available bedrock foundation models with granted access.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]

        # 只保留 access == "GRANTED" 的
        granted_models = [model for model in models if model.get("access") == "GRANTED"]

        logger.info("Got %s granted foundation models.", len(granted_models))
        return granted_models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise

def main():
    """Entry point for the example. Uses the AWS SDK for Python (Boto3)
    to create an Amazon Bedrock client. Then lists the available Bedrock models
    in the region set in the callers profile and credentials.
    """

    bedrock_client = boto3.client(service_name="bedrock")

    fm_models = list_granted_foundation_models(bedrock_client)
    for model in fm_models:
        print(f"Model: {model['modelName']}")
        print(json.dumps(model, indent=2))
        print("---------------------------\n")

    logger.info("Done.")


if __name__ == "__main__":
    main()

## AWS sample code

In [ ]:
# Use the native inference API to send a text message to Amazon Titan Text G1 - Express.

import boto3
import json

from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = brt.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]
print(response_text)

## langchain_aws

In [ ]:
from langchain_aws import BedrockLLM

model_id = "amazon.nova-lite-v1:0"

# 建立一個 Bedrock LLM 實例，指定 model_id
llm = BedrockLLM(
    model_id=model_id,
    model_kwargs={
        "temperature": 0.7,
        "maxTokenCount": 512,
        "topP": 0.9,
        "stopSequences": [],
    }
)

# 測試呼叫
response = llm.invoke("請用中文回答，介紹一下你自己。")
print(response)

## Nova & Claude

In [ ]:
# Use the Conversation API to send a text message to Amazon Nova.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Amazon Nova Lite.
model_id = "amazon.nova-lite-v1:0"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)




## S3

In [ ]:
import boto3

s3_client = boto3.client('s3', 
                  region_name='us-east-1')


def list_buckets():
    response = s3_client.list_buckets()
    buckets = response['Buckets']
    [print(f"Bucket Name: {bucket['Name']}") for bucket in buckets]
    return buckets
        

list_buckets()


In [ ]:
def list_objects(bucket_name, prefix=""):
    response = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix=prefix  # 可選：只列出特定開頭的
    )
    print(f"Objects in bucket '{bucket_name}':")
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("No objects found.")

# get all objects in all bucket
buckets = list_buckets()
for bucket in buckets:
    list_objects(bucket['Name'])

In [ ]:
import boto3

s3_client = boto3.client('s3', 
                  region_name='us-east-1')

def upload_file_to_s3(bucket_name, file_path, object_name):
    try:
        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"File '{file_path}' uploaded to bucket '{bucket_name}' as '{object_name}'.")
    except Exception as e:
        print(f"Error uploading file: {e}")

# upload folder to s3
def upload_folder_to_s3(bucket_name, folder_path):
    import os
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            object_name = os.path.relpath(file_path, folder_path)
            upload_file_to_s3(bucket_name, file_path, object_name)

upload_folder_to_s3('aiwave-hackathon-team-8', '../data')

## Bedrock Knowledge base

In [ ]:
REGION = 'us-east-1'  # 你的 AWS region
S3_BUCKET_ARN = 'arn:aws:s3:::your-s3-bucket-name'
S3_PREFIX = 'your/s3/folder/'  # S3裡PDF的資料夾，記得最後加斜線/
BEDROCK_ROLE_ARN = 'arn:aws:iam::your-account-id:role/your-bedrock-kb-role'
EMBEDDING_MODEL_ARN = f'arn:aws:bedrock:{REGION}::foundation-model/amazon.titan-embed-text-v1'

COLLECTION_NAME = 'your-collection-name'
KB_NAME = 'your-knowledge-base-name'


# ====== 建立 Knowledge Base ======
bedrock_agent = boto3.client('bedrock-agent', region_name=REGION)

print("[2] 建立 Knowledge Base...")
create_kb_response = bedrock_agent.create_knowledge_base(
    name=KB_NAME,
    description="Knowledge Base created from S3 PDFs",
    roleArn=BEDROCK_ROLE_ARN,
    knowledgeBaseConfiguration={
        "type": "VECTOR",
        "vectorKnowledgeBaseConfiguration": {
            "embeddingModelArn": EMBEDDING_MODEL_ARN
        }
    },
    storageConfiguration={
        "type": "OPENSEARCH_SERVERLESS",
        "opensearchServerlessConfiguration": {
            "collectionArn": collection_arn,
            "vectorIndexName": "bedrock-kb-vector-index",  # 可以自己改名稱
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "metadata"
            }
        }
    }
)
knowledge_base_id = create_kb_response['knowledgeBase']['knowledgeBaseId']
print(f"Knowledge Base 建立成功，ID: {knowledge_base_id}")